---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    
    # reading in test data
    blight_train = pd.read_csv('readonly/train.csv', encoding='cp1252')
    blight_test = pd.read_csv('readonly/test.csv', encoding='cp1252')
    addresses = pd.read_csv('readonly/addresses.csv')
    latlons = pd.read_csv('readonly/latlons.csv')
    
    ## CLEANING TRAINING DATA
    
    # merge 1: setting indices to ticket id
    blight_train = blight_train.set_index('ticket_id')
    addresses = addresses.set_index('ticket_id')

    # merge 1: merging on ticket id
    blight_train = pd.merge(blight_train, addresses, how = 'inner',
                        left_index=True, right_index=True)

    # resetting index, to retain ticket id in data
    blight_train = blight_train.reset_index()

    # merge 2: setting address to upper case
    blight_train['address'] = blight_train['address'].str.upper()
    latlons['address'] = latlons['address'].str.upper()

    # merge 2: setting indices to address
    blight_train = blight_train.set_index('address')
    latlons = latlons.set_index('address')

    # merge 2: merging on address
    blight_train = pd.merge(blight_train, latlons, how = 'inner',
                        left_index=True, right_index=True)

    # resetting index
    blight_train = blight_train.reset_index()
    blight_train.head()

    # Dropping all variables only in training set
    no_feat = ['balance_due', 'collection_status', 'compliance_detail', 'payment_amount', 'payment_date', 'payment_status']
    blight_train.drop(no_feat, axis=1, inplace=True)
    blight_train.head()
    
    # Dropping all unused categorical features
    cat_feat = ['address', 'city', 'disposition', 'grafitti_status', 'hearing_date', 'mailing_address_str_name', 'non_us_str_code', 'ticket_issued_date', 'violation_code', 'violation_street_name', 'violator_name', 'zip_code']
    blight_train.drop(cat_feat, axis=1, inplace=True)
    blight_train.head()
    
    # Dropping all unused continuous features
    cont_feat = ['clean_up_cost', 'mailing_address_str_number', 'violation_street_number', 'violation_zip_code']
    blight_train.drop(cont_feat, axis=1, inplace=True)
    blight_train.head()
    
    # data types: string to float
    blight_train[["fine_amount", "admin_fee", "state_fee", "late_fee", "discount_amount", "judgment_amount", "compliance", "lat", "lon"]] = blight_train[["fine_amount", "admin_fee", "state_fee", "late_fee", "discount_amount", "judgment_amount", "compliance", "lat", "lon"]].apply(pd.to_numeric, errors='coerce')
    
    # replacing null values for state with 'MI', after verifying using address 
    blight_train['state'] = np.where(blight_train['state'].isnull(), 'MI', blight_train['state'])
    
    # replacing remaining nan's with 0 in dataframe
    blight_train = blight_train.fillna(0)
    
    # Get one hot encoding of column Agency Name
    one_hot = pd.get_dummies(blight_train['agency_name'])
    # Drop column Agency Name as it is now encoded
    blight_train = blight_train.drop('agency_name',axis = 1)
    # Join the encoded df
    blight_train = blight_train.join(one_hot)
    
    # Get one hot encoding of column Inspector Name
    one_hot = pd.get_dummies(blight_train['inspector_name'])
    # Drop column Inspector Name as it is now encoded
    blight_train = blight_train.drop('inspector_name',axis = 1)
    # Join the encoded df
    blight_train = blight_train.join(one_hot)
    
    # Get one hot encoding of column State
    one_hot = pd.get_dummies(blight_train['state'])
    # Drop column state as it is now encoded
    blight_train = blight_train.drop('state',axis = 1)
    # Join the encoded df
    blight_train = blight_train.join(one_hot)
    
    # Get one hot encoding of column Country
    one_hot = pd.get_dummies(blight_train['country'])
    # Drop column country as it is now encoded
    blight_train = blight_train.drop('country',axis = 1)
    # Join the encoded df
    blight_train = blight_train.join(one_hot)
    
    # cleaning violation description
    blight_train['violation_description2'] = ''
    blight_train.loc[blight_train.violation_description2 == '', 'violation_description2'] = blight_train.violation_description.str.split().str.get(0)
    blight_train.drop(['violation_description'], axis=1, inplace=True)
    
    # Get one hot encoding of column Violation Description
    one_hot = pd.get_dummies(blight_train['violation_description2'])
    # Drop column violation description 2 as it is now encoded
    blight_train = blight_train.drop('violation_description2',axis = 1)
    # Join the encoded df
    blight_train = blight_train.join(one_hot)
    
    ## CLEANING TEST DATA
    
    # merge 1: setting indices to ticket id
    blight_test = blight_test.set_index('ticket_id')
    # addresses = addresses.set_index('ticket_id')

    # merge 1: merging on ticket id
    blight_test = pd.merge(blight_test, addresses, how = 'inner',
                            left_index=True, right_index=True)

    # resetting index, to retain ticket id in data
    blight_test = blight_test.reset_index()

    # merge 2: setting address to upper case
    blight_test['address'] = blight_test['address'].str.upper()

    # merge 2: setting indices to address
    blight_test = blight_test.set_index('address')

    # merge 2: merging on address
    blight_test = pd.merge(blight_test, latlons, how = 'inner',
                            left_index=True, right_index=True)

    # resetting index
    blight_test = blight_test.reset_index()
    blight_test.head()

    # Dropping all unused categorical features
    cat_feat = ['address', 'city', 'disposition', 'grafitti_status', 'hearing_date', 'mailing_address_str_name', 'non_us_str_code', 'ticket_issued_date', 'violation_code', 'violation_street_name', 'violator_name', 'zip_code']
    blight_test.drop(cat_feat, axis=1, inplace=True)
    blight_test.head()
    
    # Dropping all unused continuous features
    cont_feat = ['clean_up_cost', 'mailing_address_str_number', 'violation_street_number', 'violation_zip_code']
    blight_test.drop(cont_feat, axis=1, inplace=True)
    blight_test.head()

    # data types: string to numeric
    blight_test[["fine_amount", "admin_fee", "state_fee", "late_fee", "discount_amount", "judgment_amount", "lat", "lon"]] = blight_test[["fine_amount", "admin_fee", "state_fee", "late_fee", "discount_amount", "judgment_amount", "lat", "lon"]].apply(pd.to_numeric, errors='coerce')
    
    # replacing null values for state with 'MI', after verifying using address 
    blight_test['state'] = np.where(blight_test['state'].isnull(), 'MI', blight_test['state'])
    
    # replacing remaining nan's with 0 in dataframe
    blight_test = blight_test.fillna(0)
    
    # Get one hot encoding of column Agency Name
    one_hot = pd.get_dummies(blight_test['agency_name'])
    # Drop column Agency Name as it is now encoded
    blight_test = blight_test.drop('agency_name',axis = 1)
    # Join the encoded df
    blight_test = blight_test.join(one_hot)
    
    # Get one hot encoding of column Inspector Name
    one_hot = pd.get_dummies(blight_test['inspector_name'])
    # Drop column Inspector Name as it is now encoded
    blight_test = blight_test.drop('inspector_name',axis = 1)
    # Join the encoded df
    blight_test = blight_test.join(one_hot)
    
    # Get one hot encoding of column State
    one_hot = pd.get_dummies(blight_test['state'])
    # Drop column state as it is now encoded
    blight_test = blight_test.drop('state',axis = 1)
    # Join the encoded df
    blight_test = blight_test.join(one_hot)
    
    # Get one hot encoding of column Country
    one_hot = pd.get_dummies(blight_test['country'])
    # Drop column country as it is now encoded
    blight_test = blight_test.drop('country',axis = 1)
    # Join the encoded df
    blight_test = blight_test.join(one_hot)

    # cleaning violation description
    blight_test['violation_description2'] = ''
    blight_test.loc[blight_test.violation_description2 == '', 'violation_description2'] = blight_test.violation_description.str.split().str.get(0)
    blight_test.drop(['violation_description'], axis=1, inplace=True)
    
    # Get one hot encoding of column Violation Description
    one_hot = pd.get_dummies(blight_test['violation_description2'])
    # Drop column violation description 2 as it is now encoded
    blight_test = blight_test.drop('violation_description2',axis = 1)
    # Join the encoded df
    blight_test = blight_test.join(one_hot)
    
    # FINAL MODEL
    
    # train test split
    from sklearn.model_selection import train_test_split

    features = blight_train.drop('compliance', axis=1)
    labels = blight_train['compliance']

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
    
    from sklearn.metrics import roc_curve, auc
    from sklearn.linear_model import LogisticRegression
    
    # fitting model
    lr = LogisticRegression().fit(X_train, y_train)
    lr_predicted = lr.predict(X_test)
    
    # evaluating
    y_score_lr = lr.fit(X_train, y_train).decision_function(X_test)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    
    # predicting probabilities of labels
    y_proba_lr = lr.fit(X_train, y_train).predict_proba(X_test)
    y_proba_list = list(zip(y_test[0:20], y_proba_lr[0:20,1]))
    
    df = pd.DataFrame(y_proba_lr, X_test['ticket_id'])
    
    # PREDICTIONS
    
    # checking for overlapping columns
    intersection = list(set(list(blight_train.columns)).intersection(list(blight_test.columns)))
    
    # subsetting test and train data to overlapping columns
    features = blight_train.drop('compliance', axis=1)
    features = features[intersection]                                                    
    labels = blight_train['compliance']

    blight_test = blight_test[intersection]
    
    # fitting model
    lr = LogisticRegression().fit(features, labels)
    lr_predicted = lr.predict(blight_test)
    
    # evaluating
    y_score_lr = lr.fit(features, labels).decision_function(blight_test)
    
    # predicting probabilities of labels
    y_proba_lr = lr.fit(features, labels).predict_proba(blight_test)
    y_proba_list = list(zip(lr_predicted[0:20], y_proba_lr[0:20,1]))
  
    df = pd.DataFrame(y_proba_lr, blight_test['ticket_id'])
    
    return df[1].head(20)

In [2]:
blight_model()

C:\Users\dsingh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\dsingh\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dsingh\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale t

ticket_id
328722    4.577332e-02
350971    4.563717e-02
344821    4.573468e-02
358133    9.541342e-02
309054    9.615833e-02
286216    2.452008e-03
324319    4.576373e-02
355752    4.551974e-02
314897    2.659682e-01
349403    2.273533e-01
303732    5.553074e-02
303731    9.634031e-02
327945    4.558696e-02
330126    1.635342e-01
371204    1.365847e-01
371203    1.625372e-01
288692    4.598757e-02
288756    1.657030e-08
309575    4.586069e-02
311449    1.750224e-02
Name: 1, dtype: float64